# Notebook MLOlympiad for Students - TopVistos EUA - Kaggle
https://www.kaggle.com/competitions/ml-olympiad-for-students-topvistos-eua/data

In [ ]:
# Conectar no google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Importar o DataSet do google drive
df_sample_submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Kaggle/Competitions/ML Olympiad for Students - TopVistos EUA/sample_submission.csv')
df_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Kaggle/Competitions/ML Olympiad for Students - TopVistos EUA/test.csv')
df_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Kaggle/Competitions/ML Olympiad for Students - TopVistos EUA/train.csv')

# Importar as Bibliotecas

In [9]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, OneHotEncoder
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import lightgbm as lgb
import tensorflow as tf
import torch

# Arquivos do Projeto

## Dataframe Exemplo de Submissão (df_sample_submission)

In [ ]:
df_sample_submission.head()

,id_do_caso,status_do_caso
0,EZYV01,0
1,EZYV02,1
2,EZYV03,0
3,EZYV04,0
4,EZYV05,1


## Dataframe Teste (df_test)

In [ ]:
df_test.head()

,id_do_caso,continente,educacao_do_empregado,tem_experiencia_de_trabalho,requer_treinamento_de_trabalho,num_de_empregados,ano_de_estabelecimento,regiao_de_emprego,salario_prevalecente,unidade_de_salario,posicao_em_tempo_integral
0,EZYV22339,Ásia,Ensino Superior,S,N,2414,1973,Nordeste,103320.80,Ano,N
1,EZYV9333,Ásia,Mestrado,N,S,2928,1937,Oeste,49786.35,Ano,S
2,EZYV9740,Ásia,Ensino Superior,S,N,1836,1999,Oeste,60855.83,Ano,S
3,EZYV3328,Ásia,Ensino Superior,S,N,122,2004,Sul,82379.06,Semana,S
4,EZYV12321,América do Sul,Ensino Superior,S,S,1720,2003,Nordeste,6719.81,Ano,S


## Dataframe Treino (df_train)

In [8]:
df_train

,id_do_caso,continente,educacao_do_empregado,tem_experiencia_de_trabalho,requer_treinamento_de_trabalho,num_de_empregados,ano_de_estabelecimento,regiao_de_emprego,salario_prevalecente,unidade_de_salario,posicao_em_tempo_integral,status_do_caso
0,EZYV10567,Europa,Ensino Médio,N,S,2087,1855,Sul,69711.24,Ano,S,Negado
1,EZYV5505,Ásia,Mestrado,S,N,5991,2003,Meio-Oeste,52931.38,Ano,S,Aprovado
2,EZYV5207,Ásia,Ensino Médio,N,N,1426,2000,Ilha,110830.21,Ano,S,Negado
3,EZYV7545,Ásia,Ensino Médio,N,N,3846,1992,Meio-Oeste,91884.68,Semana,S,Negado
4,EZYV16071,Ásia,Ensino Superior,S,N,3957,1949,Nordeste,138155.24,Ano,S,Aprovado
...,...,...,...,...,...,...,...,...,...,...,...,...
17831,EZYV17821,Ásia,Mestrado,S,N,95347,1995,Oeste,69692.24,Ano,S,Aprovado
17832,EZYV13200,Europa,Ensino Médio,N,N,1483,1886,Sul,136237.62,Ano,S,Negado
17833,EZYV8903,América do Norte,Mestrado,N,N,1504,2010,Nordeste,118187.30,Ano,S,Negado
17834,EZYV6163,América do Norte,Doutorado,S,N,251967,2010,Meio-Oeste,93133.40,Ano,S,Aprovado


# Pré-processamento dos dados de treino (df_train)
Isso pode incluir a transformação de variáveis categóricas em numéricas, normalização/escalonamento de variáveis numéricas, codificação one-hot, etc.

In [121]:
# Definindo uma nova variável para o dataset principal não ser corrompido
df_balanced = df_train

# Selecionar as colunas categóricas
categorical_columns = ['tem_experiencia_de_trabalho', 'requer_treinamento_de_trabalho', 'posicao_em_tempo_integral']

# Aplicar a codificação one-hot nas colunas categóricas
df_balanced_encoded = pd.get_dummies(df_balanced, columns=categorical_columns)

# Separar os recursos (features) e o rótulo (target)
X = df_balanced_encoded.drop('status_do_caso', axis=1)
y = df_balanced_encoded['status_do_caso']

In [122]:
df_balanced_encoded

,id_do_caso,continente,educacao_do_empregado,num_de_empregados,ano_de_estabelecimento,regiao_de_emprego,salario_prevalecente,unidade_de_salario,status_do_caso,tem_experiencia_de_trabalho_N,tem_experiencia_de_trabalho_S,requer_treinamento_de_trabalho_N,requer_treinamento_de_trabalho_S,posicao_em_tempo_integral_N,posicao_em_tempo_integral_S
0,EZYV10567,Europa,Ensino Médio,2087,1855,Sul,69711.24,Ano,Negado,1,0,0,1,0,1
1,EZYV5505,Ásia,Mestrado,5991,2003,Meio-Oeste,52931.38,Ano,Aprovado,0,1,1,0,0,1
2,EZYV5207,Ásia,Ensino Médio,1426,2000,Ilha,110830.21,Ano,Negado,1,0,1,0,0,1
3,EZYV7545,Ásia,Ensino Médio,3846,1992,Meio-Oeste,91884.68,Semana,Negado,1,0,1,0,0,1
4,EZYV16071,Ásia,Ensino Superior,3957,1949,Nordeste,138155.24,Ano,Aprovado,0,1,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17831,EZYV17821,Ásia,Mestrado,95347,1995,Oeste,69692.24,Ano,Aprovado,0,1,1,0,0,1
17832,EZYV13200,Europa,Ensino Médio,1483,1886,Sul,136237.62,Ano,Negado,1,0,1,0,0,1
17833,EZYV8903,América do Norte,Mestrado,1504,2010,Nordeste,118187.30,Ano,Negado,1,0,1,0,0,1
17834,EZYV6163,América do Norte,Doutorado,251967,2010,Meio-Oeste,93133.40,Ano,Aprovado,0,1,1,0,0,1


In [123]:
# deletar as colunas categóricas_N
del df_balanced_encoded['tem_experiencia_de_trabalho_N']
del df_balanced_encoded['requer_treinamento_de_trabalho_N']
del df_balanced_encoded['posicao_em_tempo_integral_N']

# Selecionar as colunas categóricas
categorical_columns = ['status_do_caso']

# Aplicar a codificação one-hot nas colunas categóricas
df_balanced_encoded = pd.get_dummies(df_balanced_encoded, columns=categorical_columns)

# deletando a coluna status_do_caso_Negado
del df_balanced_encoded['status_do_caso_Negado']

df_balanced_encoded

,id_do_caso,continente,educacao_do_empregado,num_de_empregados,ano_de_estabelecimento,regiao_de_emprego,salario_prevalecente,unidade_de_salario,tem_experiencia_de_trabalho_S,requer_treinamento_de_trabalho_S,posicao_em_tempo_integral_S,status_do_caso_Aprovado
0,EZYV10567,Europa,Ensino Médio,2087,1855,Sul,69711.24,Ano,0,1,1,0
1,EZYV5505,Ásia,Mestrado,5991,2003,Meio-Oeste,52931.38,Ano,1,0,1,1
2,EZYV5207,Ásia,Ensino Médio,1426,2000,Ilha,110830.21,Ano,0,0,1,0
3,EZYV7545,Ásia,Ensino Médio,3846,1992,Meio-Oeste,91884.68,Semana,0,0,1,0
4,EZYV16071,Ásia,Ensino Superior,3957,1949,Nordeste,138155.24,Ano,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
17831,EZYV17821,Ásia,Mestrado,95347,1995,Oeste,69692.24,Ano,1,0,1,1
17832,EZYV13200,Europa,Ensino Médio,1483,1886,Sul,136237.62,Ano,0,0,1,0
17833,EZYV8903,América do Norte,Mestrado,1504,2010,Nordeste,118187.30,Ano,0,0,1,0
17834,EZYV6163,América do Norte,Doutorado,251967,2010,Meio-Oeste,93133.40,Ano,1,0,1,1


Listando os valores únicos de cada coluna

In [124]:
df_balanced_encoded.info

<bound method DataFrame.info of       id_do_caso        continente educacao_do_empregado  num_de_empregados  \
0      EZYV10567            Europa          Ensino Médio               2087   
1       EZYV5505              Ásia              Mestrado               5991   
2       EZYV5207              Ásia          Ensino Médio               1426   
3       EZYV7545              Ásia          Ensino Médio               3846   
4      EZYV16071              Ásia       Ensino Superior               3957   
...          ...               ...                   ...                ...   
17831  EZYV17821              Ásia              Mestrado              95347   
17832  EZYV13200            Europa          Ensino Médio               1483   
17833   EZYV8903  América do Norte              Mestrado               1504   
17834   EZYV6163  América do Norte             Doutorado             251967   
17835   EZYV4219              Ásia       Ensino Superior               4116   

       ano_de_estab

In [125]:
# Listando os valores únicos de cada coluna
df_balanced_encoded.nunique()

id_do_caso                          17836
continente                              6
educacao_do_empregado                   4
num_de_empregados                    6294
ano_de_estabelecimento                199
regiao_de_emprego                       5
salario_prevalecente                17827
unidade_de_salario                      4
tem_experiencia_de_trabalho_S           2
requer_treinamento_de_trabalho_S        2
posicao_em_tempo_integral_S             2
status_do_caso_Aprovado                 2
dtype: int64

In [126]:
# Listando os valores únicos da coluna continente
df_balanced_encoded['continente'].unique()

array(['Europa', 'Ásia', 'América do Sul', 'América do Norte', 'África',
       'Oceania'], dtype=object)

In [127]:
# Listando os valores únicos da coluna educacao_do_empregado
df_balanced_encoded['educacao_do_empregado'].unique()

array(['Ensino Médio', 'Mestrado', 'Ensino Superior', 'Doutorado'],
      dtype=object)

In [128]:
# Listando os valores únicos da coluna regiao_de_emprego
df_balanced_encoded['regiao_de_emprego'].unique()

array(['Sul', 'Meio-Oeste', 'Ilha', 'Nordeste', 'Oeste'], dtype=object)

In [129]:
# Listando os valores únicos da coluna unidade_de_salario
df_balanced_encoded['unidade_de_salario'].unique()

array(['Ano', 'Semana', 'Hora', 'Mês'], dtype=object)

In [130]:
# Transformando valores únicos de strings em valores numéricos
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df_balanced_encoded['continente_encoded'] = label_encoder.fit_transform(df_balanced_encoded['continente'])

label_encoder = LabelEncoder()
df_balanced_encoded['educacao_do_empregado_encoded'] = label_encoder.fit_transform(df_balanced_encoded['educacao_do_empregado'])

label_encoder = LabelEncoder()
df_balanced_encoded['regiao_de_emprego_encoded'] = label_encoder.fit_transform(df_balanced_encoded['regiao_de_emprego'])

label_encoder = LabelEncoder()
df_balanced_encoded['unidade_de_salario_encoded'] = label_encoder.fit_transform(df_balanced_encoded['unidade_de_salario'])

In [131]:
print(df_balanced_encoded['continente_encoded'].unique())

[2 5 1 0 4 3]


In [132]:
print(df_balanced_encoded['educacao_do_empregado_encoded'].unique())

[1 3 2 0]


In [133]:
print(df_balanced_encoded['regiao_de_emprego_encoded'].unique())

[4 1 0 2 3]


In [134]:
print(df_balanced_encoded['unidade_de_salario_encoded'].unique())

[0 3 1 2]


In [135]:
df_balanced_encoded

,id_do_caso,continente,educacao_do_empregado,num_de_empregados,ano_de_estabelecimento,regiao_de_emprego,salario_prevalecente,unidade_de_salario,tem_experiencia_de_trabalho_S,requer_treinamento_de_trabalho_S,posicao_em_tempo_integral_S,status_do_caso_Aprovado,continente_encoded,educacao_do_empregado_encoded,regiao_de_emprego_encoded,unidade_de_salario_encoded
0,EZYV10567,Europa,Ensino Médio,2087,1855,Sul,69711.24,Ano,0,1,1,0,2,1,4,0
1,EZYV5505,Ásia,Mestrado,5991,2003,Meio-Oeste,52931.38,Ano,1,0,1,1,5,3,1,0
2,EZYV5207,Ásia,Ensino Médio,1426,2000,Ilha,110830.21,Ano,0,0,1,0,5,1,0,0
3,EZYV7545,Ásia,Ensino Médio,3846,1992,Meio-Oeste,91884.68,Semana,0,0,1,0,5,1,1,3
4,EZYV16071,Ásia,Ensino Superior,3957,1949,Nordeste,138155.24,Ano,1,0,1,1,5,2,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17831,EZYV17821,Ásia,Mestrado,95347,1995,Oeste,69692.24,Ano,1,0,1,1,5,3,3,0
17832,EZYV13200,Europa,Ensino Médio,1483,1886,Sul,136237.62,Ano,0,0,1,0,2,1,4,0
17833,EZYV8903,América do Norte,Mestrado,1504,2010,Nordeste,118187.30,Ano,0,0,1,0,0,3,2,0
17834,EZYV6163,América do Norte,Doutorado,251967,2010,Meio-Oeste,93133.40,Ano,1,0,1,1,0,0,1,0


In [136]:
# Delentando colunas
df_balanced_encoded_filtered = df_balanced_encoded

del df_balanced_encoded_filtered['continente']
del df_balanced_encoded_filtered['educacao_do_empregado']
del df_balanced_encoded_filtered['regiao_de_emprego']
del df_balanced_encoded_filtered['unidade_de_salario']

df_balanced_encoded_filtered

,id_do_caso,num_de_empregados,ano_de_estabelecimento,salario_prevalecente,tem_experiencia_de_trabalho_S,requer_treinamento_de_trabalho_S,posicao_em_tempo_integral_S,status_do_caso_Aprovado,continente_encoded,educacao_do_empregado_encoded,regiao_de_emprego_encoded,unidade_de_salario_encoded
0,EZYV10567,2087,1855,69711.24,0,1,1,0,2,1,4,0
1,EZYV5505,5991,2003,52931.38,1,0,1,1,5,3,1,0
2,EZYV5207,1426,2000,110830.21,0,0,1,0,5,1,0,0
3,EZYV7545,3846,1992,91884.68,0,0,1,0,5,1,1,3
4,EZYV16071,3957,1949,138155.24,1,0,1,1,5,2,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...
17831,EZYV17821,95347,1995,69692.24,1,0,1,1,5,3,3,0
17832,EZYV13200,1483,1886,136237.62,0,0,1,0,2,1,4,0
17833,EZYV8903,1504,2010,118187.30,0,0,1,0,0,3,2,0
17834,EZYV6163,251967,2010,93133.40,1,0,1,1,0,0,1,0


In [137]:
# Transformando a coluna ano_de_estabelecimento para anos_estabelecimento
# calculando a diferença entre o ano atual e o ano de criação do estabelecimento
ano_atual = 2023
df_balanced_encoded_filtered['anos_estabelecimento'] = ano_atual - df_balanced_encoded_filtered['ano_de_estabelecimento']
del df_balanced_encoded_filtered['ano_de_estabelecimento']
df_balanced_encoded_filtered

,id_do_caso,num_de_empregados,salario_prevalecente,tem_experiencia_de_trabalho_S,requer_treinamento_de_trabalho_S,posicao_em_tempo_integral_S,status_do_caso_Aprovado,continente_encoded,educacao_do_empregado_encoded,regiao_de_emprego_encoded,unidade_de_salario_encoded,anos_estabelecimento
0,EZYV10567,2087,69711.24,0,1,1,0,2,1,4,0,168
1,EZYV5505,5991,52931.38,1,0,1,1,5,3,1,0,20
2,EZYV5207,1426,110830.21,0,0,1,0,5,1,0,0,23
3,EZYV7545,3846,91884.68,0,0,1,0,5,1,1,3,31
4,EZYV16071,3957,138155.24,1,0,1,1,5,2,2,0,74
...,...,...,...,...,...,...,...,...,...,...,...,...
17831,EZYV17821,95347,69692.24,1,0,1,1,5,3,3,0,28
17832,EZYV13200,1483,136237.62,0,0,1,0,2,1,4,0,137
17833,EZYV8903,1504,118187.30,0,0,1,0,0,3,2,0,13
17834,EZYV6163,251967,93133.40,1,0,1,1,0,0,1,0,13


In [138]:
# Movendo a coluna unidade_de_salario_encoded para a posição 3

# Obter a lista de todas as colunas do DataFrame
colunas = df_balanced_encoded_filtered.columns.tolist()

# Remover a coluna 'unidade_de_salario' do seu índice original
colunas.remove('unidade_de_salario_encoded')

# Inserir a coluna 'unidade_de_salario' na nova posição (índice 3)
colunas.insert(3, 'unidade_de_salario_encoded')

# Reordenar as colunas do DataFrame com a nova ordem
df_balanced_encoded_filtered = df_balanced_encoded_filtered[colunas]

# Mostrando a nova ordem
df_balanced_encoded_filtered

,id_do_caso,num_de_empregados,salario_prevalecente,unidade_de_salario_encoded,tem_experiencia_de_trabalho_S,requer_treinamento_de_trabalho_S,posicao_em_tempo_integral_S,status_do_caso_Aprovado,continente_encoded,educacao_do_empregado_encoded,regiao_de_emprego_encoded,anos_estabelecimento
0,EZYV10567,2087,69711.24,0,0,1,1,0,2,1,4,168
1,EZYV5505,5991,52931.38,0,1,0,1,1,5,3,1,20
2,EZYV5207,1426,110830.21,0,0,0,1,0,5,1,0,23
3,EZYV7545,3846,91884.68,3,0,0,1,0,5,1,1,31
4,EZYV16071,3957,138155.24,0,1,0,1,1,5,2,2,74
...,...,...,...,...,...,...,...,...,...,...,...,...
17831,EZYV17821,95347,69692.24,0,1,0,1,1,5,3,3,28
17832,EZYV13200,1483,136237.62,0,0,0,1,0,2,1,4,137
17833,EZYV8903,1504,118187.30,0,0,0,1,0,0,3,2,13
17834,EZYV6163,251967,93133.40,0,1,0,1,1,0,0,1,13


Normalizando o salario_prevalecente

In [139]:
df_balanced_encoded_filtered['salario_prevalecente'].dtype

dtype('float64')

In [140]:
df_balanced_encoded_filtered['unidade_de_salario_encoded'].dtype

dtype('int64')

In [147]:
# Criar uma cópia do DataFrame original
df_balanced_encoded_filtered_normal = df_balanced_encoded_filtered.copy()

# Aplicar as modificações na coluna 'salario_prevalecente' com base na coluna 'unidade_de_salario_encoded'
df_balanced_encoded_filtered_normal['salario_prevalecente'] = np.where(df_balanced_encoded_filtered_normal['unidade_de_salario_encoded'] == 0,
                                                                      df_balanced_encoded_filtered_normal['salario_prevalecente'] / 48,
                                                                      df_balanced_encoded_filtered_normal['salario_prevalecente'])

df_balanced_encoded_filtered_normal['salario_prevalecente'] = np.where(df_balanced_encoded_filtered_normal['unidade_de_salario_encoded'] == 1,
                                                                      df_balanced_encoded_filtered_normal['salario_prevalecente'] * 96,
                                                                      df_balanced_encoded_filtered_normal['salario_prevalecente'])

df_balanced_encoded_filtered_normal['salario_prevalecente'] = np.where(df_balanced_encoded_filtered_normal['unidade_de_salario_encoded'] == 2,
                                                                      df_balanced_encoded_filtered_normal['salario_prevalecente'] / 4,
                                                                      df_balanced_encoded_filtered_normal['salario_prevalecente'])

df_balanced_encoded_filtered_normal['salario_prevalecente'] = np.where(df_balanced_encoded_filtered_normal['unidade_de_salario_encoded'] == 3,
                                                                      df_balanced_encoded_filtered_normal['salario_prevalecente'] * 1,
                                                                      df_balanced_encoded_filtered_normal['salario_prevalecente'])

df_balanced_encoded_filtered_normal

,id_do_caso,num_de_empregados,salario_prevalecente,unidade_de_salario_encoded,tem_experiencia_de_trabalho_S,requer_treinamento_de_trabalho_S,posicao_em_tempo_integral_S,status_do_caso_Aprovado,continente_encoded,educacao_do_empregado_encoded,regiao_de_emprego_encoded,anos_estabelecimento
0,EZYV10567,2087,1452.317500,0,0,1,1,0,2,1,4,168
1,EZYV5505,5991,1102.737083,0,1,0,1,1,5,3,1,20
2,EZYV5207,1426,2308.962708,0,0,0,1,0,5,1,0,23
3,EZYV7545,3846,91884.680000,3,0,0,1,0,5,1,1,31
4,EZYV16071,3957,2878.234167,0,1,0,1,1,5,2,2,74
...,...,...,...,...,...,...,...,...,...,...,...,...
17831,EZYV17821,95347,1451.921667,0,1,0,1,1,5,3,3,28
17832,EZYV13200,1483,2838.283750,0,0,0,1,0,2,1,4,137
17833,EZYV8903,1504,2462.235417,0,0,0,1,0,0,3,2,13
17834,EZYV6163,251967,1940.279167,0,1,0,1,1,0,0,1,13


In [148]:
# Mudando o nome da coluna salario_prevalecente para salario_prevalecente_semanal
df_balanced_encoded_filtered_normal.rename(columns={'salario_prevalecente': 'salario_prevalecente_semanal'}, inplace=True)

# Delentando a coluna unidade_de_salario_encoded
del df_balanced_encoded_filtered_normal['unidade_de_salario_encoded']

# Arredonando a coluna salario_prevalecente_semanal para 2 casas decimais
df_balanced_encoded_filtered_normal['salario_prevalecente_semanal'] = df_balanced_encoded_filtered_normal['salario_prevalecente_semanal'].round(2)

In [149]:
# Renomeando o dataframe para df_normal
df_normal = df_balanced_encoded_filtered_normal
df_normal

,id_do_caso,num_de_empregados,salario_prevalecente_semanal,tem_experiencia_de_trabalho_S,requer_treinamento_de_trabalho_S,posicao_em_tempo_integral_S,status_do_caso_Aprovado,continente_encoded,educacao_do_empregado_encoded,regiao_de_emprego_encoded,anos_estabelecimento
0,EZYV10567,2087,1452.32,0,1,1,0,2,1,4,168
1,EZYV5505,5991,1102.74,1,0,1,1,5,3,1,20
2,EZYV5207,1426,2308.96,0,0,1,0,5,1,0,23
3,EZYV7545,3846,91884.68,0,0,1,0,5,1,1,31
4,EZYV16071,3957,2878.23,1,0,1,1,5,2,2,74
...,...,...,...,...,...,...,...,...,...,...,...
17831,EZYV17821,95347,1451.92,1,0,1,1,5,3,3,28
17832,EZYV13200,1483,2838.28,0,0,1,0,2,1,4,137
17833,EZYV8903,1504,2462.24,0,0,1,0,0,3,2,13
17834,EZYV6163,251967,1940.28,1,0,1,1,0,0,1,13


# Trabalhando com o novo dataframe (df_normal) balanceado e normalizado

## Salvando o novo dataframe (df_normal) balanceado e normalizado em um arquivo .csv

In [150]:
# Salvando o novo dataframe (df_normal) balanceado e normalizado em um arquivo .csv
# df_normal.to_csv('df_normal.csv', index=False)

In [153]:
df_normal = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Kaggle/Competitions/ML Olympiad for Students - TopVistos EUA/df_normal.csv')
df_normal

,id_do_caso,num_de_empregados,salario_prevalecente_semanal,tem_experiencia_de_trabalho_S,requer_treinamento_de_trabalho_S,posicao_em_tempo_integral_S,status_do_caso_Aprovado,continente_encoded,educacao_do_empregado_encoded,regiao_de_emprego_encoded,anos_estabelecimento
0,EZYV10567,2087,1452.32,0,1,1,0,2,1,4,168
1,EZYV5505,5991,1102.74,1,0,1,1,5,3,1,20
2,EZYV5207,1426,2308.96,0,0,1,0,5,1,0,23
3,EZYV7545,3846,91884.68,0,0,1,0,5,1,1,31
4,EZYV16071,3957,2878.23,1,0,1,1,5,2,2,74
...,...,...,...,...,...,...,...,...,...,...,...
17831,EZYV17821,95347,1451.92,1,0,1,1,5,3,3,28
17832,EZYV13200,1483,2838.28,0,0,1,0,2,1,4,137
17833,EZYV8903,1504,2462.24,0,0,1,0,0,3,2,13
17834,EZYV6163,251967,1940.28,1,0,1,1,0,0,1,13


## Analisando o dataframe (df_normal)

In [ ]:
# correlação, etc